# `QuantumPioneer/databases` Demo
This notebook demonstrates how to interact with the `QuantumPioneer` databases. It does:
1) reads the QP DLPNO demo database from the local directory.
2) creates a full and short CSV version 
3) creates an OWL ABOX file based on an OWL file created from the DLPNO schema
4) it uploads the OWL ABOX file to a local tripple store - blazegraph
5) performs various SPARQL queries on the small data base. 

See inline comments for more information.

In [1]:
import numpy as np

# both pandas and polars can read parquet files - pick whichever you prefer!
# there are advantages and disadvantages to both
import pandas as pd
import polars as pl

import pyarrow as pa
# this library interacts with the parquet format directly, and both pandas and polars can use it too
import pyarrow.parquet as pq

# schema = layout of the database (what are the datatypes, etc.)
# the schema for the quantumpioneer databases are stored in databases.schema and vary depending on the type of data
from databases.schema import DLPNO_SCHEMA, DFT_SCHEMA

# import libraries for KG handling
from owlready2 import *
import csv
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD, OWL
import os
import requests


## DLPNO

In [2]:
# set this to match wherever you have the database file located
# DLPNO_DATABASE_FPATH = "QuantumPioneer_v1_DLPNO.parquet"  original code Jackson
# DLPNO_DATABASE_FPATH = "C:\\VS mk306 TestEnv\\QuantumPioneer\\databases\\dlpno_ts_v5.parquet"    mk306 Jackson folder 
DLPNO_DATABASE_FPATH = "C:\\VS mk306 TestEnv\\Codes\\mk_PracticeEnv\\dlpno_ts_v5.parquet"

In [5]:
# just open and read the entire dataset (very practical with the DLPNO data, which is small), which will be slow with pandas
df = pd.read_parquet(
    DLPNO_DATABASE_FPATH,
    schema=DLPNO_SCHEMA,  # pandas will try and guess this on its own if you don't provide it - it gets it right, but is slower
)
df.head(4)

# Optionally view the first 4 rows
print(df.head(4))

# Export the DataFrame to a CSV file without the index
output_csv = 'myQPCSVhead10.csv'
df.head(10).to_csv(output_csv, index=False)
# df.to_csv(output_csv, index=False)
print(f"Data successfully exported to {output_csv}")





                                              source  \
0  /data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...   
1  /data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...   
2  /data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...   
3  /data1/groups/RMG/Projects/Hao-Wei-Oscar-Yunsi...   

                                       route_section  charge  multiplicity  \
0  uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...       0             2   
1  uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...       0             2   
2  uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...       0             2   
3  uHF UNO DLPNO-CCSD(T)-F12D cc-pvtz-f12 def2/J ...       0             2   

       energy  run_time                                  input_coordinates  \
0 -552.285416    2401.0  [[-1.18079, 1.675527, 1.578389], [-0.208623, 2...   
1 -570.750611    2936.0  [[2.188448, 0.23618, -1.582947], [1.462597, -0...   
2 -572.077854    2755.0  [[1.863694, -1.837741, 0.404006], [1.711148, -...   
3 -572.156665   

In [6]:
# write CSV to OWL

def csv_to_owl(csv_file, owl_tbox_file, output_owl_file):
    try:
        # Verify files exist
        if not os.path.exists(csv_file):
            raise FileNotFoundError(f"CSV file not found: {csv_file}")
        if not os.path.exists(owl_tbox_file):
            raise FileNotFoundError(f"OWL TBox file not found: {owl_tbox_file}")

        # Initialize RDF Graph
        g = Graph()
        
        # Parse the existing TBox ontology
        print(f"Loading ontology from: {owl_tbox_file}")
        g.parse(owl_tbox_file, format='xml')
        
        # Define namespaces - using the same URI as in your ontology file
        base_ns = Namespace("http://example.org/ontologies/DLPNO.owl#")
        g.bind("dlpno", base_ns)
        
        # Read CSV file and create individuals
        print(f"Processing CSV file: {csv_file}")
        with open(csv_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            
            for i, row in enumerate(reader, 1):
                try:
                    print(f"Processing row {i}")
                    # Create a unique URI for each individual
                    individual_uri = URIRef(base_ns[f"result_{i}"])
                    
                    # Add type assertion
                    g.add((individual_uri, RDF.type, base_ns.DLPNOResult))
                    
                    # Add properties from CSV with proper error handling
                    properties = [
                        ('source', 'hasSource', XSD.string, str),
                        ('route_section', 'hasRouteSection', XSD.string, str),
                        ('charge', 'hasCharge', XSD.unsignedByte, int),
                        ('multiplicity', 'hasMultiplicity', XSD.unsignedByte, int),
                        ('energy', 'hasEnergy', XSD.double, float),
                        ('run_time', 'hasRunTime', XSD.float, float),
                        ('dipole_au', 'hasDipoleAU', XSD.float, float),
                        ('t1_diagnostic', 'hasT1Diagnostic', XSD.float, float)
                    ]
                    
                    for csv_col, prop_name, xsd_type, type_func in properties:
                        if csv_col in row:
                            try:
                                value = type_func(row[csv_col]) if row[csv_col] else None
                                if value is not None:
                                    prop_uri = URIRef(base_ns + prop_name)
                                    g.add((individual_uri, prop_uri, Literal(value, datatype=xsd_type)))
                            except (ValueError, TypeError) as e:
                                print(f"Warning: Could not process {csv_col} value '{row[csv_col]}': {str(e)}")
                    
                    # Special handling for coordinates
                    if 'input_coordinates' in row:
                        coords = row['input_coordinates'].strip('"\'')
                        g.add((individual_uri, base_ns.hasInputCoordinates, 
                              Literal(coords, datatype=XSD.string)))
                
                except Exception as e:
                    print(f"Error processing row {i}: {str(e)}")
                    continue
        
        # Serialize the complete ontology
        print(f"Saving output to: {output_owl_file}")
        g.serialize(destination=output_owl_file, format='xml')
        print(f"Successfully created OWL file with ABox at: {output_owl_file}")
        
    except Exception as e:
        print(f"Fatal error: {str(e)}")
        raise

# Usage with your actual filenames
csv_to_owl(
    csv_file='myQPCSVhead10.csv',
    owl_tbox_file='ontoQP-DPLO.owl',
    output_owl_file='ontoQP-DPLO_abox.owl'
)

Loading ontology from: ontoQP-DPLO.owl
Processing CSV file: myQPCSVhead10.csv
Processing row 1
Processing row 2
Processing row 3
Processing row 4
Processing row 5
Processing row 6
Processing row 7
Processing row 8
Processing row 9
Processing row 10
Saving output to: ontoQP-DPLO_abox.owl
Successfully created OWL file with ABox at: ontoQP-DPLO_abox.owl


In [7]:
# upload the OWL file to local tripple store (Blazegraph) 
# # Blazegraph endpoint - can only read
# blazegraph_url = "http://localhost:3838/blazegraph/namespace/kb/sparql"

# Blazegraph endpoint can also write
blazegraph_url = "http://localhost:3838/blazegraph/namespace/kb/update"


# Path to your RDF file
rdf_file_path = r"C:\VS mk306 TestEnv\Codes\mk_PracticeEnv\ontoQP-DPLO_abox.owl"


# Headers for the request
# headers = {
#     "Content-Type": "text/turtle"
# }
headers = {
    "Content-Type": "application/rdf+xml"
}

# Read the RDF file
with open(rdf_file_path, "r") as file:
    rdf_data = file.read()

# Send the data to Blazegraph
response = requests.post(blazegraph_url, headers=headers, data=rdf_data)

# Check the response
if response.status_code == 200:
    print("Data loaded successfully!")
else:
    print(f"Failed to load data. Status code: {response.status_code}")
    print(response.text)




Data loaded successfully!


In [8]:
# Perform SPRQL query on local blazegraph 
# Blazegraph SPARQL Query Endpoint
sparql_endpoint = blazegraph_url

#--- SPARQL Query  - find all triples
sparql_query = """
SELECT ?s ?p ?o
WHERE {
    ?s ?p ?o .
} LIMIT 2
"""

# Send HTTP request
response = requests.get(sparql_endpoint, params={"query": sparql_query, "format": "json"})

# Parse JSON response
results = response.json()

# Print results
for result in results["results"]["bindings"]:
    print(result["s"]["value"], result["p"]["value"], result["o"]["value"])


# --- Query 1: Retrieve all DLPNO results and their total energies.
sparql_query = """
PREFIX dl: <http://example.org/ontologies/DLPNO.owl#>

SELECT ?result ?energy
WHERE {
  ?result a dl:DLPNOResult ;
          dl:hasEnergy ?energy .
}
ORDER BY ?energy
LIMIT 3
"""

# Send HTTP request
response = requests.get(
    sparql_endpoint,
    params={"query": sparql_query, "format": "json"}
)
response.raise_for_status()  # Check for HTTP errors

# Parse JSON response
results = response.json()

# Print results (use correct variable names: ?result, ?energy)
for binding in results["results"]["bindings"]:
    print(
        "Result:", binding["result"]["value"],
        "| Energy:", binding["energy"]["value"]
    )


# --- Query 2: Find DLPNO calculations with runtime longer than 5000 seconds.
sparql_query = """
PREFIX dl: <http://example.org/ontologies/DLPNO.owl#>

SELECT ?result ?runtime
WHERE {
  ?result a dl:DLPNOResult ;
          dl:hasRunTime ?runtime .
  FILTER(?runtime > 2500)
}
ORDER BY DESC(?runtime)
LIMIT 3
"""

# Send HTTP request
response = requests.get(
    sparql_endpoint,
    params={"query": sparql_query, "format": "json"}
)
response.raise_for_status()  # Check for HTTP errors

# Parse JSON response
results = response.json()

# Print results (corrected variable names: ?result, ?runtime)
for binding in results["results"]["bindings"]:
    print(
        "Result:", binding["result"]["value"],
        "| Runtime:", binding["runtime"]["value"]
    )


# --- Query 3: Extract DLPNO results by multiplicity and charge.
sparql_query = """
PREFIX dl: <http://example.org/ontologies/DLPNO.owl#>

SELECT ?result ?multiplicity ?charge
WHERE {
  ?result a dl:DLPNOResult ;
          dl:hasMultiplicity ?multiplicity ;
          dl:hasCharge ?charge .
}
ORDER BY ?charge ?multiplicity
LIMIT 3
"""

# Send HTTP request
response = requests.get(
    sparql_endpoint,
    params={"query": sparql_query, "format": "json"}
)
response.raise_for_status()  # Check for HTTP errors

# Parse JSON response
results = response.json()

# Print results - CORRECTED VARIABLES
for binding in results["results"]["bindings"]:
    print(
        "Result:", binding["result"]["value"],
        "| Multiplicity:", binding["multiplicity"]["value"],
        "| Charge:", binding["charge"]["value"]
    )

# --- Query 4: Find DLPNO results with high T1 diagnostic values (> 0.02)
sparql_query = """
PREFIX dl: <http://example.org/ontologies/DLPNO.owl#>

SELECT ?result ?t1Diagnostic
WHERE {
  ?result a dl:DLPNOResult ;
          dl:hasT1Diagnostic ?t1Diagnostic .
  FILTER(?t1Diagnostic > 0.02)
}
ORDER BY DESC(?t1Diagnostic)
LIMIT 3
"""

# Send HTTP request
response = requests.get(
    sparql_endpoint,
    params={"query": sparql_query, "format": "json"}
)
response.raise_for_status()  # Check for HTTP errors

# Parse JSON response
results = response.json()

# Print results - CORRECTED to use actual selected variables
for binding in results["results"]["bindings"]:
    print(
        "Result:", binding["result"]["value"],
        "| T1 Diagnostic:", binding["t1Diagnostic"]["value"]
    )


# --- Query 5: Find DLPNO results with source and coordinates   
sparql_query = """
PREFIX dl: <http://example.org/ontologies/DLPNO.owl#>

SELECT ?result ?source ?coordinates
WHERE {
  ?result a dl:DLPNOResult ;
          dl:hasSource ?source ;
          dl:hasInputCoordinates ?coordinates .
}
LIMIT 1
"""

# Send HTTP request
response = requests.get(
    sparql_endpoint,
    params={"query": sparql_query, "format": "json"}
)
response.raise_for_status()  # Check for HTTP errors

# Parse JSON response
results = response.json()

# Print results - Needs to be improved 
for binding in results["results"]["bindings"]:
    print(
        "Result:", binding["result"]["value"],
        "| Source:", binding["source"]["value"],
        "| Coordinates:", binding["coordinates"]["value"]
    )


# --- Query 6: Find DLPNO results with specific route sections

http://example.org/ontologies/DLPNO.owl#result_5 http://example.org/ontologies/DLPNO.owl#hasCharge 0
http://example.org/ontologies/DLPNO.owl#result_5 http://example.org/ontologies/DLPNO.owl#hasDipoleAU 0.85609
Result: http://example.org/ontologies/DLPNO.owl#result_9 | Energy: -611.72942540614
Result: http://example.org/ontologies/DLPNO.owl#result_6 | Energy: -589.360925465229
Result: http://example.org/ontologies/DLPNO.owl#result_8 | Energy: -575.799460863625
Result: http://example.org/ontologies/DLPNO.owl#result_10 | Runtime: 3488.0
Result: http://example.org/ontologies/DLPNO.owl#result_2 | Runtime: 2936.0
Result: http://example.org/ontologies/DLPNO.owl#result_8 | Runtime: 2803.0
Result: http://example.org/ontologies/DLPNO.owl#result_1 | Multiplicity: 2 | Charge: 0
Result: http://example.org/ontologies/DLPNO.owl#result_10 | Multiplicity: 2 | Charge: 0
Result: http://example.org/ontologies/DLPNO.owl#result_2 | Multiplicity: 2 | Charge: 0
Result: http://example.org/ontologies/DLPNO.owl#

## Here are Jackson's output examples for the parquet files

In [ ]:
# from here you can do all of your usual pandas manipulations
df.iloc[0][["route_section", "input_coordinates"]].to_list()

In [ ]:
# you can reduce the amount of memory consumed by only loading the columns that you care about using columns=...
df = pd.read_parquet(DLPNO_DATABASE_FPATH, columns=["source", "energy"])
df.head(4)

In [ ]:
# and you filter out specific rows _when reading_ the database to further reduce memory consumption (and speed things up)
# these statements can be complex, but the pandas docs explain it well:
# https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html
df = pd.read_parquet(
    DLPNO_DATABASE_FPATH,
    # SKIP rows where...
    filters=[
        [
            (  # multiplicity is equal to 1
                "multiplicity",
                "=",
                1,
            ),  # AND
            (  # energy is less than -500
                "energy",
                "<",
                -500,
            ),
        ],
        [  # OR
            (  # these two specific files
                "source",
                "not in",
                (
                    "/data1/groups/co2_capture/reactant_product_calculation/ts_nho_round1/output/DLPNO_sp_f12/outputs/outputs_146/146857.log",
                    "/data1/groups/co2_capture/reactant_product_calculation/ts_nho_round1/output/DLPNO_sp_f12/outputs/outputs_146/146989.log",
                ),
            ),
        ],
    ],
)
df.head(4)

In [ ]:
# you can also filter out based on one row without actually loading it
df = pd.read_parquet(DLPNO_DATABASE_FPATH, filters=[("run_time", "<", 100)], columns=["source", "charge"])
df.head(4)

In [ ]:
# you can run everything from above using polars as well, and in my experience it uses less memory and is faster
df = pl.read_parquet(DLPNO_DATABASE_FPATH)
df.head(4)

In [ ]:
# a notable difference is that polars sets `memory_map=True` by default (pandas supports it, but is False and accessible via kwarg only)

In [ ]:
# to pass filters to polars, you have to use the `pyarrow_options` argument (polars only supports limiting the number of rows in
# in sequential order via `n_rows`)
df = pl.read_parquet(
    DLPNO_DATABASE_FPATH,
    columns=["source", "charge"],
    pyarrow_options=dict(
        filters=[("run_time", "<", 100)],
        schema=DLPNO_SCHEMA,
    ),
)
df.head(4)

In [ ]:
# ...or just use polars other functions
df = (
    pl.scan_parquet(
        DLPNO_DATABASE_FPATH,
    )  # opens the file, but does not actually read it (LazyFrame)
    .filter(
        pl.col("run_time") < 100,
    )  # sets up our filters, but still does not run the query
    .select(pl.col("source"), pl.col("energy"))
    .collect()  # actually runs the query
)
df.head(4)

In [ ]:
# the final option is to interact with the data via pyarrow directly, which takes all the same arguments as before but in a slightly
# different setup - this is the single fastest way to read the data
table = pq.ParquetDataset(DLPNO_DATABASE_FPATH, schema=DLPNO_SCHEMA, filters=[("run_time", "<", 100)]).read(columns=["source", "energy"])
df = table.to_pandas()
df.head(4)